## Build a QnA Agent and Promote to LLM Model Registry
Author: `Max.AI`

This document presents a detailed overview of building agent using Max LLM Components.

**Components**:
- **Purpose and Design**: The MaxAgentQA is crafted for question-answering tasks. It captures the advanced capabilities of retrieval-augmented generation, which combines the retrieval of relevant information with sophisticated language generation models.
 - **Configuration Requirements**: To effectively utilize this agent, the user needs to provide specific configurations. These include settings for the Language Large Model (LLM), VectorDB, MaxFlow Configuration. The LLM configuration would define how the model processes and generates language responses, while the VectorDB configuration would pertain to how the system handles and retrieves vectorized data representations.The MaxFlow configuration would enable monitoring and logging.
- **Key Methods**:
> - `add()`: This method is likely designed for adding or indexing new data to the agent. It involves processing, splitting, and storing documents, questions, or other relevant data in a format that can be efficiently retrieved and used by the language model for answering questions.
> - `query()`: This function enables the user to input queries or questions and retrieve answers. It leverages the combined power of the configured LLM, VectorDB, MaxRetriever, and MaxGenerator to search for relevant information and generate accurate, contextually appropriate responses.

**Significance and Application**:
The MaxAgentQA is a pivotal component for a Data scientist looking to build an efficient and intelligent question-answering agent. By integrating retrieval and generation capabilities, it offers a more dynamic and context-aware approach to handling queries. 

### Step 1 : Initialize the Environment Variables from Config Store

The Config Store contains the essential configurations like - 
- **LLM API Key**: This includes the OPEN_AI_API_KEY, Azure OpenAI, Anthropic Model etc details.
- **Vector Database Details**: These details captures the database's user, password, name, port, and other relevant information which are required for setting up and connecting to the vector database.
- **MLflow Credentials** : These configurations enables the Max Monitoring capability, using this each individual steps like parameters, artifacts like models, configurations, metrices can be logged and montitored using MLFlow UI

In [1]:
from maxairesources.config_store.config_store import ConfigStore
config_store = ConfigStore(secrets_manager='max/agents')

### Step 2 : MaxFlow - Setting up Monitoring and Logginng

**MaxFlow** - 
We have integrated advanced logging and monitoring features into Max.AI through the MaxFlow accelerator. This pre-built component is designed to meticulously track the entire process of building an agent, here for an instance Question & Answering Agent.

Key aspects of MaxFlow include:

- **Detailed Logging**: MaxFlow logs every experiment at a granular level. This includes recording each step of the process, ensuring comprehensive documentation of the QnA agent's development.
- **Parameter Tracking**: It captures and logs all parameters used in the process, providing clarity on how the agent is configured at each stage.
- **Artifact Management**: MaxFlow handles artifacts like models and configurations, logging their usage and changes. This helps in maintaining a track of all components used in the QnA agent’s development.
- **Metric Monitoring**: The tool is equipped to log various metrics, offering insights into the performance and efficiency of the QnA agent.
- **MLFlow UI Integration**: All these aspects parameters, artifacts, and metrics are accessible and can be monitored through the MLFlow User Interface. This integration allows for an intuitive and user-friendly way to review and analyze the logged information.

In [ ]:
from maxaimetadata.maxflow import MaxFlow
mf = MaxFlow(uri = config_store.config_data["MAXFLOW_URI"] )

In [3]:
mf.set_experiment(experiment='Experiment-Name',repo_path='.')
#create a new run by passing Run name, Type and Description
max_run = mf.start_run(name = 'QnA', 
                   type = 'QA',
                   description = 'Max QA Agents')

{"timestamp":"2024-01-30 15:36:01,432","pid":"2031","level":"INFO","file_name":"1787171647.py","hierarchy":"maxaimetadata.maxflow","function_name":"<cell line: 1>","line_number":"1","message":"Successfully set expriment Jan30newEnterpriseQnADemo","extra":null, "experiment": "NA","run": "NA","max_version": "NA"}
{"timestamp":"2024-01-30 15:36:01,500","pid":"2031","level":"INFO","file_name":"1787171647.py","hierarchy":"maxaimetadata.maxflow","function_name":"<cell line: 3>","line_number":"3","message":"Active run run_id: 22de4a4708a94dcfb2495199a9cb7fb8","extra":null, "experiment": "Jan30newEnterpriseQnADemo","run": "22de4a4708a94dcfb2495199a9cb7fb8","max_version": "NA"}
{"timestamp":"2024-01-30 15:36:01,501","pid":"2031","level":"INFO","file_name":"1787171647.py","hierarchy":"maxaimetadata.maxflow","function_name":"<cell line: 3>","line_number":"3","message":"Active run name: QnA","extra":null, "experiment": "Jan30newEnterpriseQnADemo","run": "22de4a4708a94dcfb2495199a9cb7fb8","max_vers

### Step 3 : Initialize the MaxAgentQA

Here we focus on initializing the MaxAgentQA, a component designed specifically for question-answering tasks.
The MaxAgentQA can be fully configurable and can accept the below parameters

**Parameter list**
- **llm (LLM)**: An instance of a Language Learning Model to be used for text processing.
- **chunk_size** (int, optional): The size of text chunks for processing. Defaults to 2000.
- **chunk_overlap** (int, optional): The overlap size between consecutive text chunks. Defaults to 200.
- **stream** (bool, optional): Flag to indicate if the data should be processed as a stream. Defaults to False.
- **collection** (str, optional): The name of the collection to be used in the vector database. If None, initializes later.
- **prompt_config** (Dict[str, str], optional): Configuration settings for prompts. If None, an error is raised.
- **metadata** (Dict[str, bool], optional): A dictionary to specify which metadata features to include. Defaults to a predefined set.
- **embedding_model** (str, optional): The name of the embedding model to be used. Defaults to "sentence-transformers/all-mpnet-base-v2".
- **retriever_rank** (str, optional): The ranking method to be used by the retriever. Defaults to "LostInMiddle".
- **generate_method** (str, optional): The method used for generation tasks. Defaults to "stuff".
- **verbose**(bool, optional) : The parameter sets the verbose parameter for the QA chain

In [4]:
from maxaillm.agents.MaxAgentQA import MaxAgentQA

agent_qa = MaxAgentQA(
                llm_provider = "anthropic",
                model_name = "claude-2",
                prompt_config = {'moderations':'','task':'','identity':'', 'output_format': ''}, 
                collection = "Collection-Name", 
                verbose = True, 
                vector_store = 'pgvector',
                metadata_dict = {'default' : True,
                                "frequent_keywords" : False, 
                                "links" : True})


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Vector DB initialized
Loaded Anthropic model: claude-2


/opt/conda/lib/python3.9/site-packages/langchain/llms/anthropic.py:180: UserWarning: This Anthropic LLM is deprecated. Please use `from langchain.chat_models import ChatAnthropic` instead
  warnings.warn(


#### Step 3.1 : `add()` method

It plays a crucial role in processing and managing documents within the MaxAgentQA

**Functionalities of add()**:
- **Processing a List of Documents**: The method starts by taking in a list of documents. These documents can be in various formats and contain different types of content. Here we use `MaxExtractor`, which parses the files and converts the unstructured formatted document into structured plain text.

- **Extracting and Splitting Document Content**: For each document in the list, add() performs extraction and splitting operations. Here we leverage `MaxSplitter`. This involves  splitting this content into smaller chunks based on the document type and semantic understanding. This step is critical for making the data more accessible and easier to analyze.

- **Storing Processed Data in VectorDB**: After processing, the `MaxEmbeddings` module stores the resulting data in the vector database using `MaxVectorization`, which is essential for efficient data retrieval and analysis in various machine learning and NLP tasks.

- **Return Status**: The method concludes by returning a boolean value indicating the success of the process. It returns True if the processing and storage of documents were successful. Conversely, it returns False if any errors were encountered during the process.


In [ ]:
paths = [
  'path/to/sample.pdf'
]

agent_qa.add(paths)
max_run.log_datasets(paths)

#### Step 3.2 Call Method `query()`

It processes a input user query and retrieves the answer along with references.

**Functionality of query()**:
- **Processing an Input Query**: The method begins by taking an input query from the user. This query is typically in the form of a natural language question or request.

- **Retrieving the Answer**: Upon receiving the query, query() processes it to understand its context and intent. The query is passed to `MaxRetriever` which then searches through the stored data in the VectorDB to find the most relevant and accurate answer to the query. The module retrieves relevant chunks for a particular query based on the chosen vector search algorithm. The retrieved chunks are then passed to `MaxGenerator` to generate the output for the user in the desired format.
Along with the answer, the method also provides references or sources from which the answer was derived. This is an important aspect, as it adds credibility to the response and allows users to verify the information or explore the topic further.

- **Return Format**: The output from this method is the textual response for the query, along with associated references, in the required user format for the user’s understanding and further exploration.

In [ ]:
resp = agent_qa.query(query= 'Summarize 2022 performance')
resp

### Step 4 Logging the MaxAgentQA

- Here we are leveraging MaxFlow logging capability to log the Agent we built above, this agent can be loaded and leveraged by the developer to build applications.

In [ ]:
max_run.log_agent(agent_qa)

max_run.end_run()

In [ ]:
mf.stop()

### The MaxAgentQA is designed to be easily integrated into different software applications. 
